The goal of this script is to log the RGB Camera, Depth Camera, and Force Data at the same sampling rate and time via the serial port. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import cv2
import serial
import time 

# Setup:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)


# Create an object to read  
# from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 



# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
colorizer = rs.colorizer()
colorizer.set_option(rs.option.color_scheme, 3)
mag_deci = 4
decimation = rs.decimation_filter(mag_deci)
threshold = rs.threshold_filter(0.15, 1.6)

n_trial = 30
depthwriter = cv2.VideoWriter('fcss_leo_depth_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'DIVX'), 
                         30, (int(frame_width/mag_deci), int(frame_height/mag_deci)), 0) 


print("type anything to start")
start_test = input()


addr = "COM4" ## serial port to read data from
baud = 115200 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud)

fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
datafile = open(fcss_filename, 'a')




# Start streaming
pipe.start(cfg)
start_time = cv2.getTickCount()
prev_time = 0
count = 0
count_total = 0 
try:
    while ((cv2.getTickCount() - start_time)/cv2.getTickFrequency() < 15):
        try:
            prev_time = cv2.getTickCount()
            ser_bytes = ser.readline()
            decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))

#             print(decoded_bytes)

            write_line = ",".join(decoded_bytes) + str("\n")
            datafile.write(write_line)
            
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipe.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            
            if not depth_frame:
                continue
            
            # Decimation 
            depth_frame = decimation.process(depth_frame)

            # Thresholding
            depth_frame = threshold.process(depth_frame)
        
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())     
            key = cv2.waitKey(1)
            
            # Resize the CV frame to fit the decimated frame
            depth_save = cv2.resize(depth_colormap, (int(frame_width/mag_deci), int(frame_height/mag_deci)))
            
            
 
            # Save depth camera frame 
            depthwriter.write(depth_save)
            
            # Show depth camera 
            cv2.imshow('Depth', depth_save)

            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
                depthwriter.release()
                break 
            count_total = count_total+ 1
            
            el_t = (cv2.getTickCount() - prev_time)/cv2.getTickFrequency()
            
            if el_t > 0.04:
                count = count + 1
                print(el_t)
            prev_time = cv2.getTickCount()
            
                
        except:
            pass
            break

finally:
    ser.close()
    datafile.close()
    pipe.stop()
    cv2.destroyAllWindows()
    video.release() 
    depthwriter.release()
    print(count/count_total * 100)

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
# config.enable_record_to_file('object_detection.bag')

# Start streaming
pipeline.start(config)
e1 = cv2.getTickCount()


count = 0
count_total = 0 

try:
    while True:
        e11 = cv2.getTickCount()
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not color_frame or not depth_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        e2 = cv2.getTickCount()
        t = (e2 - e1) / cv2.getTickFrequency()
        t1 = (e2 - e11) / cv2.getTickFrequency()

        count_total = count_total+ 1
        if t1 > 0.033:
            count = count + 1
        if t > 10: # change it to record what length of video you are interested in
            print('Done')
            break

finally:

    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    print(count/count_total*100)

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import serial


# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start streaming
pipeline.start(config)

# Convert to black-to-white color scheme
colorizer = rs.colorizer()
colorizer.set_option(rs.option.color_scheme, 3)


# Filter parameters
mag_deci = 4
decimation = rs.decimation_filter(mag_deci)
threshold = rs.threshold_filter(0.15, 1.6)



# Create an object to read from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
size = (frame_width, frame_height) 


# Save Video
n_trial = 30
depthwriter = cv2.VideoWriter('fcss_leo_depth_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'DIVX'), 
                         30, (int(frame_width/mag_deci), int(frame_height/mag_deci)), 0) 



# arduino_port = "COM4" #serial port of Arduino
# baud = 115200 #arduino uno runs at 9600 baud
# fileName="test_1.csv" #name of the CSV file generated
# ser = serial.Serial(arduino_port, baud)
# print("Connected to Arduino port:" + arduino_port)
# file = open(fileName, "a")
# print("Created file")

# print_labels = False
# line = 0 #start at 0 because our header is 0 (not real data)

e1 = cv2.getTickCount()

count = 0
count_total = 0 

try:
    while True:
        e11 = cv2.getTickCount()
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        if not depth_frame:
            continue

        # Decimation 
        depth_frame = decimation.process(depth_frame)

        # Thresholding
        depth_frame = threshold.process(depth_frame)
            
        # Apply colormap on depth image (image must be converted to8-bit per pixel first)
        depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())  
        
        # Show images
        cv2.imshow('RealSense', depth_colormap)
        cv2.waitKey(1)
                
#         if print_labels:
#             if line==0:
#                 print("Printing Column Headers")
#             else:
#                 print("Line " + str(line) + ": writing...")
#         getData=str(ser.readline())
#         data=getData[0:][:-2]
#         print(data)

#         file = open(fileName, "a")
#         file.write(data + "\\n") #write data with a newline
#         line = line+1
        
#         key = cv2.waitKey(1)
#         if key & 0xFF == ord('q') or key == 27:
#                 cv2.destroyAllWindows()
#                 video.release() 
#                 depthwriter.release()
#                 break 
                
        e2 = cv2.getTickCount()
        t = (e2 - e1) / cv2.getTickFrequency()
        t1 = (e2 - e11) / cv2.getTickFrequency()

        count_total = count_total+ 1
        if t1 > 0.033:
            count = count + 1
        if t > 5: # change it to record what length of video you are interested in
            print('Done')
            break

finally:

    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    video.release() 
    print("Data collection complete!")
    ser.close()
    file.close()
    print(count/count_total * 100)
    

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import serial 

n_trial = 31

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
bag_filename = 'bag_test' + str(n_trial) + '.bag'
config.enable_record_to_file(bag_filename)

# addr = "COM4" 
# baud = 115200 
# ser = serial.Serial(
#     port = addr,\
#     baudrate = baud)

fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
datafile = open(fcss_filename, 'a')

# Start streaming
pipeline.start(config)
count = 0
count_total = 0
e1 = cv2.getTickCount()

try:
    while True:
        e11 = cv2.getTickCount()

#         ser_bytes = ser.readline()
#         decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))
#         write_line = ",".join(decoded_bytes) + str("\n")
#         datafile.write(write_line)
        
        e2 = cv2.getTickCount()
        t = (e2 - e1) / cv2.getTickFrequency()
        t1 = (e2 - e11) / cv2.getTickFrequency()
        print(t1)
        
        if t1 > 0.04:
            count = count + 1
            
        count_total = count_total+ 1
        
finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    ser.close()
    datafile.close()
    print(count/count_total * 100)


In [ ]:
import time
import multiprocessing

start = time.perf_counter()

def do_something():
    print("sleep")
    time.sleep(1)
    print("done")
    


p1 = multiprocessing.Process(target = do_something)
p2 = multiprocessing.Process(target = do_something)

p1.start()
p2.start()

finish = time.perf_counter()

print(f'Finished in {round(finish - start, 2)} seconds')

In [ ]:
a = time.perf_counter()
a

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import cv2
import serial
import time 

# Setup Realsense environment
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 424, 240, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)


# Create an object to read from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 



# Changing colormap to "black-white" 
colorizer = rs.colorizer()
colorizer.set_option(rs.option.color_scheme, 3)

# Applying filters. Decimation filter for smoothening, Thresholding for removing background. 
mag_deci = 4
decimation = rs.decimation_filter(mag_deci) # smoothen using 4x4 kernel. Mean depth is used. 
threshold = rs.threshold_filter(0.15, 1.6) # allow only objects from 0.15m ~ 1.6m


print("type anything to start")
start_test = input()

# Start streaming
pipe.start(cfg)
start_time = cv2.getTickCount()


try:
    while True:
        try:
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipe.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            
            color_frame = frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue
            
            # Decimation 
            depth_frame = decimation.process(depth_frame)

            # Thresholding
            depth_frame = threshold.process(depth_frame)
        
            # Apply colormap on image (image must be converted to 8-bit per pixel first)
            depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())     
            color_image = np.asanyarray(color_frame.get_data())
            
            key = cv2.waitKey(1)
            
            # Resize the CV frame to fit the decimated frame
            depth_display = cv2.resize(depth_colormap, (int(frame_width/mag_deci), int(frame_height/mag_deci)))
            
            # Show depth camera 
            cv2.imshow('Depth', depth_display)

            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
                break 
   
                
        except:
            pass
            break

finally:
    pipe.stop()
    cv2.destroyAllWindows()
    video.release() 


In [10]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)


# This is for saving the video files. I highly recommend first saving the files into '.bag' file like this and then 
# post processing it to get .avi files. This is the recommended way outlined by Intel
#config.enable_record_to_file('object_detection.bag')


# Create an object to read from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 

# We need to set resolutions. so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 

    
# Changing depth colormap to "black-white" 
colorizer = rs.colorizer()
colorizer.set_option(rs.option.color_scheme, 3)

# Applying filters. Decimation filter for smoothening, Thresholding for removing background. 
mag_deci = 4
decimation = rs.decimation_filter(mag_deci) # smoothen using 4x4 kernel. Mean depth is used. 
threshold = rs.threshold_filter(0.15, 1.6) # allow only objects from 0.15m ~ 1.6m

# Start streaming
pipeline.start(config)

e1 = cv2.getTickCount()

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

    
        # Decimation
        depth_frame = decimation.process(depth_frame)

        # Thresholding
        depth_frame = threshold.process(depth_frame)

        # Apply colormap on image (image must be converted to 8-bit per pixel first)
        depth_final = np.asanyarray(colorizer.colorize(depth_frame).get_data())     
        
        # Resize rgb color map
        color_final = cv2.resize(color_image, (int(frame_width/mag_deci), int(frame_height/mag_deci)))
        
        # Stack both images horizontally
        images = np.hstack((color_final, depth_final))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        
        e2 = cv2.getTickCount()
        
        t = (e2 - e1) / cv2.getTickFrequency()
        if t>30: # change it to record what length of video you are interested in
            print("Done!")
            break
            
        
        
        # Press 'q' or 'ESC' to close the image window
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            video.release() 
            break 

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    video.release() 
